In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import mglearn
import warnings

warnings.filterwarnings('ignore') 
plt.style.use('classic')

## 이진 분류의 평가 지표
---

### 불균형 데이터
#### 클래스의 비율이 한쪽으로 치우처진 데이터를 말한다.
#### ex) 클릭된 광고(1%), 클릭되지 않은 광고(99%)

In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
digits = load_digits()
y = digits.target == 9

X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=0)

#### 실제 모델과 비교하기 위해 가단한 규칙을 지원하는 모델
#### DummyClassifier
- strategy :
    - stratified : 클래스 레이블 비율에 맞게 예측한다.
    - most_frequent : 가장 많은 레이블을 예측한다.

#### DummyRegressor 
- strategy :
    - mean : 평균 값을 예측한다.
    - median : 중간값을 예측한다.

In [5]:
# 항상 다수의 클래스를 예측하는 모델을 사용함.
from sklearn.dummy import DummyClassifier
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
pred_most_frequent = dummy_majority.predict(X_test)
print("에측된 레이블의 레이블", np.unique(pred_most_frequent))
print(f"test score : {dummy_majority.score(X_test, y_test):0.3f}")

# 모든 클래스를 False 로 예측했지만 정확도는 약 90%나 된다.

에측된 레이블의 레이블 [False]
test score : 0.896


In [8]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
pred_tree = tree.predict(X_test)
print(f"test score : {tree.score(X_test, y_test):0.3f}")

# DecisionTreeClassfier 의 성능이 조금 높다.
# 이로써 DecisionTreeClassfier 의 성능이 좋은 편이 아니라는 것을 알 수 있다.

test score : 0.918


In [16]:
from sklearn.linear_model import LogisticRegression

dummy = DummyClassifier(strategy='stratified').fit(X_train, y_train)
pred_dummy = dummy.predict(X_test)
print(f"dummy score : {dummy.score(X_test, y_test):0.3f}")

logreg = LogisticRegression(C=0.1, max_iter=1000).fit(X_train, y_train)
pred_logreg = logreg.predict(X_test)
print(f"logreg score : {logreg.score(X_test, y_test):0.3f}")

dummy score : 0.840
logreg score : 0.984
